<a href="https://colab.research.google.com/github/rahul-negi-projects/X_Education/blob/main/fake_news_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
dataset_path = "/content/drive/MyDrive/Colab_Notebooks/NLP/train.csv"

In [ ]:
df = pd.read_csv(dataset_path)

In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df.shape

(20800, 5)

In [ ]:
df = df.dropna()

In [ ]:
df.shape

(18285, 5)

In [ ]:
X = df.drop("label", axis=1)

In [ ]:
y = df.label

In [ ]:
X.shape

(18285, 4)

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

'2.5.0'

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
# vocabulary size 
voc_size = 5000

One hot represtentation

In [ ]:
messages = X.copy()

In [ ]:
messages.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [ ]:
messages.reset_index(inplace=True)

In [ ]:
messages.head()

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


Data preprocessing

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
corpus = []
for i in range(0, len(messages)):
  # print(i)
  review = re.sub("[^a-zA-Z]"," ",messages["title"][i])
  review = review.lower()
  review = review.split()
  # print(review)

  review = [ps.stem(word) for word in review if not  word in stopwords.words('english')]
  # print(review)
  review = " ".join(review)
  # print(review)
  corpus.append(review)

In [ ]:
corpus[0:2]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart']

In [ ]:
# One hot representation
onehot_repr = [one_hot(word, voc_size) for word in corpus]
onehot_repr[0:2]

[[2903, 3964, 4544, 1019, 4266, 3777, 1749, 4223, 1183, 39],
 [1967, 4526, 3417, 2680, 4557, 3856, 1609]]

In [ ]:
# padding to make the length of all the sentences equal
sent_len = 20

embeded_docs = pad_sequences(onehot_repr, padding="pre", maxlen=sent_len)
embeded_docs[0:2]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 2903,
        3964, 4544, 1019, 4266, 3777, 1749, 4223, 1183,   39],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0, 1967, 4526, 3417, 2680, 4557, 3856, 1609]],
      dtype=int32)

In [ ]:
len(embeded_docs)

18285

Creating model

In [ ]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(input_dim= voc_size, output_dim= embedding_vector_features, input_length= sent_len))
model.add(LSTM(100)) # one LSTM layer with 100 neurons
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer="adam", metrics= ["accuracy"])
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
import numpy as np
X_final = np.array(embeded_docs)
y_final = np.array(y)

In [ ]:
embeded_docs[0:2]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 2903,
        3964, 4544, 1019, 4266, 3777, 1749, 4223, 1183,   39],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0, 1967, 4526, 3417, 2680, 4557, 3856, 1609]],
      dtype=int32)

In [ ]:
X_final[0:2]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 2903,
        3964, 4544, 1019, 4266, 3777, 1749, 4223, 1183,   39],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0, 1967, 4526, 3417, 2680, 4557, 3856, 1609]],
      dtype=int32)

In [ ]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

Model Training

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 3s 10ms/step - loss: 0.3404 - accuracy: 0.8475 - val_loss: 0.1943 - val_accuracy: 0.9132
Epoch 2/10
192/192 [==============================] - 2s 9ms/step - loss: 0.1327 - accuracy: 0.9481 - val_loss: 0.1901 - val_accuracy: 0.9183
Epoch 3/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0816 - accuracy: 0.9698 - val_loss: 0.2472 - val_accuracy: 0.9133
Epoch 4/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0497 - accuracy: 0.9847 - val_loss: 0.2789 - val_accuracy: 0.9044
Epoch 5/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0291 - accuracy: 0.9918 - val_loss: 0.2775 - val_accuracy: 0.9155
Epoch 6/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0164 - accuracy: 0.9948 - val_loss: 0.4291 - val_accuracy: 0.9044
Epoch 7/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0091 - accuracy: 0.9974 - val_loss: 0.4617 - val_accuracy: 0.9084
Epoch

model is overfitting

Accuracy

In [ ]:
y_pred = model.predict_classes(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score
print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test,y_pred))

[[3098  321]
 [ 245 2371]]
0.9062137531068766


Adding dropout layer

In [ ]:
from tensorflow.keras.layers import Dropout

embedding_vector_features= 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length= sent_len))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_7 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 4s 10ms/step - loss: 0.3306 - accuracy: 0.8449 - val_loss: 0.1960 - val_accuracy: 0.9150
Epoch 2/10
192/192 [==============================] - 2s 8ms/step - loss: 0.1360 - accuracy: 0.9459 - val_loss: 0.2158 - val_accuracy: 0.9181
Epoch 3/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0944 - accuracy: 0.9638 - val_loss: 0.2129 - val_accuracy: 0.9171
Epoch 4/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0708 - accuracy: 0.9745 - val_loss: 0.2274 - val_accuracy: 0.9160
Epoch 5/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0470 - accuracy: 0.9849 - val_loss: 0.2928 - val_accuracy: 0.9082
Epoch 6/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0337 - accuracy: 0.9893 - val_loss: 0.4487 - val_accuracy: 0.9127
Epoch 7/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0229 - accuracy: 0.9929 - val_loss: 0.4293 - val_accuracy: 0.9135
Epoch

In [ ]:
y_pred = model.predict_classes(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score
print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test,y_pred))

[[3094  325]
 [ 213 2403]]
0.9108533554266777
